In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
import urllib3
import requests
import json
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
table_name = ""
index_name = ""
auth_client_id = ""
auth_client_secret = ""
auth_audience = ""
auth_url = "https://sso.beta.authrock.com/oauth/token"
tokenizer_url = "https://beta-security-tokenizer.np.rocket-logic.foc.zone"


def startEncryption():
    encrypt = Encrypt()
    for client in encrypt.get_clients():
        encryptedssn = encrypt.encrypt_ssn()
        encrypt.update_encrypted_ssn(
            client["LoanNumber"], client["SortKey"], encryptedssn
        )


class Authenicated:
    __access_token = None
    __expiry_time = None

    def get_token(self):
        if (self.__access_token is None) or (self.__expiry_time <= datetime.now()):
            print("New token")
            request_body = {
                "client_id": auth_client_id,
                "client_secret": auth_client_secret,
                "audience": auth_audience,
                "grant_type": "client_credentials",
            }
            response = requests.post(sso_url, data=request_body, verify=False)
            if response.status_code == 200:
                jsonResp = response.json()
                expires_seconds = jsonResp["expires_in"]
                self.__access_token = jsonResp["access_token"]
                self.__expiry_time = datetime.now() + timedelta(seconds=expires_seconds)
                return self.__access_token
        else:
            print("reuse token")
            return self.__access_token


class Encrypt:
    def __init__(self):
        db = boto3.resource("dynamodb", region_name="us-east-2", verify=False)
        self.table = db.Table(table_name)

    def __get_client_records(self):
        last_key = None
        condition = Key("LoanIdentifier").eq("PI") & Key("SortKey").begins_with("PI#")
        while True:
            response = self.table.query(
                IndexName=index_name,
                KeyConditionExpression=condition,
                Limit=30,
                ExclusiveStartKey=last_key,
            )
            if response:
                items = response["Items"]
                for item in items:
                    ssn = item["SSN"] if "SSN" in item else ""
                    client = {
                        "LoanNumber": item["LoanNumber"],
                        "SortKey": item["SortKey"],
                        "ssn": ssn,
                    }
                    yield client
                if "LastEvaluatedKey" in response:
                    break
                last_key = response["LastEvaluatedKey"]

    def get_clients(self):
        for client in self.__get_client_records():
            ssn = client["ssn"]
            if not ssn:
                continue
            else:
                yield client

    def encrypt_ssn(self, ssn):
        url = f"{tokenizer_url}/token/ssn"
        c = Authenicated()
        data = {"value": ssn}
        header = {"Authorization": c.get_token()}
        response = requests.post(url, data, headers=header, verify=False)
        if response.status_code == 200:
            content = response.json()
            return content["token"]
        return ""

    def update_encrypted_ssn(self, loanNumber, sortKey, encryptedssn):
        try:
            self.table.update_item(
                Key={"LoanNumber": loanNumber, "SortKey": sortKey},
                UpdateExpression="set SSN=:ssn, SchemaVersion=:v",
                ExpressionAttributeValues={":ssn": encryptedssn, ":v": 4},
            )
        except ex as Exception:
            print(ex)
            # log the loan, sortkey, encryptedssn